<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA Biomass Modeling

_by Jeremy Freeman (CarbonPlan), September 19, 2020_

This notebook extracts fits predictive biomass growth curves from FIA data

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from forests import load, setup, plot, fit

In [4]:
setup.plotting()

In [48]:
df = load.biomass(store='local', states=['CA'])
df = load.biomass_features(store='local', df=df)

In [49]:
df['type_code'].unique()

array([924., 185., 368., 225., 281., 221., 262., 922., 371., 261., 241.,
       202., 201., 941., 341., 923., 911., 921., 933., 962., 369., 184.,
       974., 934., 935., 367., 931., 361., 270., 226., 365., 942., 363.,
       280., 910., 961., 901., 203., 366., 305., 912., 943., 301., 704.,
       973., 364., 383., 222., 224., 976., 342., 972., 267., 125., 709.])

In [61]:
inds = df['type_code'] == 341 # 261 is a problem, 941 / 371 is a good example, 341 shows model deviation
x = df[inds]['age']
y = df[inds]['biomass']
plot.xy(x=x, y=y, color=df['tmax'], xlim=[0, 200], ylim=[0, 500])

alt.Chart(...)

In [24]:
model = fit.growth(x=x, y=y, noise='gamma')
print(model.result)

      fun: 1741.134103696178
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-4.54747351e-05, -4.75120032e-01,  8.78576429e+00, -2.27373675e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 305
      nit: 36
   status: 0
  success: True
        x: array([1.01960581e+03, 1.04885163e-02, 1.00000000e+00, 1.03428225e+02])


In [25]:
model.r2(x, y)

0.46023065955578335

In [26]:
yhat = model.predict(x)
samples = model.sample(x)

In [27]:
xlim = [0, 400]
ylim= [0,600]
((plot.xy(x=x, y=y, xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=yhat, xlim=xlim, ylim=ylim))
|
(plot.xy(x=x, y=samples, xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=yhat, xlim=xlim, ylim=ylim)))

alt.HConcatChart(...)

In [30]:
plot.carto(lat=df['lat'], lon=df['lon'], cmap='blues', clim=(0, 2000))

alt.Chart(...)

In [ ]:
import fsspec

In [ ]:
path = setup.loading('local')

In [ ]:
state = 'CA'

In [ ]:
import pandas as pd
import xarray as xr

In [ ]:
df['INVYR'].max()

In [ ]:
df = pd.read_parquet(path / f'processed/fia-states/long/{state.lower()}.parquet')

In [ ]:
path = setup.loading('local')

In [ ]:
mapper = fsspec.get_mapper(path / 'processed/terraclimate/conus/4000m/raster.zarr')

In [ ]:
ds = xr.open_zarr(mapper)

In [ ]:
crs = ds['crs']

In [ ]:
from pyproj import transform, Proj

In [ ]:
ds['ppt']


In [ ]:
Proj?

In [ ]:
from carbonplan_data.utils import albers_conus_crs, albers_conus_extent

In [ ]:
albers_conus_extent()

In [ ]:
from rasterio import Affine

In [ ]:
t = Affine(albers_conus_extent())

In [ ]:
ds

In [ ]:
ds['crs'].to_dict()

In [ ]:
Proj(crs.to_dict())

In [ ]:
ids = df[['LAT', 'LON']].to_xarray()

In [ ]:
ds.sel(lat=ids['LAT'], lon=ids['LON'], method='nearest')